# Reading & visualizaing raw data, cropping, filtering, and saving

## Import the few packages we'll need

In [ ]:
import matplotlib
import pathlib
import mne

Ensure Matplotlib uses the `Qt5Agg` backend, which is the best choice for MNE-Python's interactive plotting functions.

In [ ]:
matplotlib.use('Qt5Agg')

Retrieve the storage location of the `sample` data, and download the dataset if it cannot be found.

In [ ]:
sample_data_dir = mne.datasets.sample.data_path()

# Convert to a pathlib.Path for more convenience
sample_data_dir = pathlib.Path(sample_data_dir)
sample_data_dir

## Load some raw data!

In [ ]:
raw_path = sample_data_dir / 'MEG' / 'sample' / 'sample_audvis_raw.fif'
raw = mne.io.read_raw(raw_path)
raw

## Let's visualize the raw data!

In [ ]:
raw.plot()

## Extract events from the `STIM` channels

In [ ]:
events = mne.find_events(raw)

In [ ]:
event_id = {
    'Auditory/Left': 1,
    'Auditory/Right': 2,
    'Visual/Left': 3,
    'Visual/Right': 4,
    'Smiley': 5,
    'Button': 32
}
event_id

In [ ]:
len(events[events[:, 2] == 32])

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>How many <strong>visual</strong> events are in the data?</li>
    </ul>
</div>

## Plot the raw data again, but add event markers

In [ ]:
raw.plot(events=events, event_id=event_id)

## Gather some info about the data

In [ ]:
raw.info

In [ ]:
raw.info['meas_date']

In [ ]:
raw.info['sfreq']

In [ ]:
raw.info['bads']

In [ ]:
raw.ch_names[:10]

In [ ]:
raw.info['chs'][0]

## Visualize the sensor locations

In [ ]:
raw.plot_sensors(ch_type='eeg')

In [ ]:
raw.plot_sensors(kind='3d', ch_type='eeg')

## Mark channels as bad

Mark an additional EEG channel as bad and view the topoplot.

In [ ]:
raw.info['bads']

In [ ]:
raw.info['bads'] += ['EEG 051']
raw.plot_sensors(ch_type='eeg')

## Select only a subset of the channels

In [ ]:
raw_eeg = raw.copy().pick_types(meg=False, eeg=True, eog=True, exclude=[])
len(raw_eeg.ch_names)

In [ ]:
raw_eeg.info

In [ ]:
raw_eeg.plot(events=events, event_id=event_id)

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Select only MEG channels ("meg")</li>
         <li>Select only magnetometer channels ("mag")</li>
    </ul>
</div>

## Crop and filter the data

In [ ]:
raw_eeg_cropped = raw_eeg.copy().crop(tmax=100)
raw_eeg_cropped.times[-1]

In [ ]:
raw_eeg_cropped_filtered = raw_eeg_cropped.filter(l_freq=0.1, h_freq=40)

In [ ]:
raw_eeg_cropped.load_data()
raw_eeg_cropped_filtered = raw_eeg_cropped.copy().filter(l_freq=0.1, h_freq=40)

In [ ]:
raw_eeg_cropped.plot(events=events, event_id=event_id)
raw_eeg_cropped_filtered.plot(events=events, event_id=event_id)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2)

raw_eeg_cropped.plot_psd(ax=ax[0], show=False)
raw_eeg_cropped_filtered.plot_psd(ax=ax[1], show=False)

ax[0].set_title('PSD before filtering')
ax[1].set_title('PSD after filtering')
ax[1].set_xlabel('Frequency (Hz)')
fig.set_tight_layout(True)
plt.show()

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Filter the raw data with a 1 Hz high-pass and a 30 Hz low-pass filter and plot the PSD.</li>
    </ul>
</div>

## Save the data

In [ ]:
raw_eeg_cropped_filtered.save(pathlib.Path('out_data') / 'eeg_cropped_filt_raw.fif', 
                              overwrite=True)